# Sentiment Analysis with LSTM

Import Dependencies + Settings

In [ ]:
%store -r initial_processing_df

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, SimpleRNN, Embedding, LSTM
#from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import confusion_matrix
#from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.models import load_model

from tqdm import tqdm
#from textblob import TextBlob
#from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.sentiment import vader
from nltk.tokenize import word_tokenize

from collections import Counter


import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import keras
#import random
import nltk
import re

np.random.seed(124)
tf.random.set_seed(124)

plt.style.use('ggplot')

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Training Dataset Preprocessing

In [ ]:
# Loading the data
df = pd.read_csv('../Examensarbete_DS21/rnn_hotel_training_data.csv', encoding='ISO-8859-1')
df.shape

In [ ]:
# Let's have a look at our dataset
df.head()

In [ ]:
# We only need the review text and raiting
df = df[['Review Text', 'Review Rating']]
df

In [ ]:
# Checking for null values.
df.isna().any()

## Positive Column

In [ ]:
# If rating [1, 2, 3] = Negative and if rating [4, 5] = Positive
def ratings(rating):
    if rating>0 and rating<=3:
        return 0
    if rating>3 and rating<=5:
        return 1
    
df['Positive'] = df['Review Rating'].apply(ratings)
df.head()

## Word Count Column

In [ ]:
def word_count(review):
    review_list = review.split()
    return len(review_list)

df['Word_count'] = df['Review Text'].apply(word_count)
df.head()

# NLTK Text Processing Function

In [ ]:
def preprocess_text(sen):
    '''Cleans up text data, leaving 2 or more char long non-stopwords containing A-Z & a-z only in lowercase'''

    sentence = sen.lower()
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    
    # Remove single characters that are not part of words with more than one character
    sentence = re.sub(r"\b[a-zA-Z]\b", '', sentence)
    
    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = sentence.split()
    filtered_words = [word for word in words if not word in stop_words]
    
    return ' '.join(filtered_words)


# EDA

In [ ]:
df.describe()

## Count of Reviews by Stars

In [ ]:
df['Review Rating'].value_counts()

In [ ]:
ax = df['Review Rating'].value_counts().sort_index().plot(
    kind='bar',
    title='Count of Reviews by Stars',
    figsize=(8, 4)
)
ax.set_xlabel('Review Stars')
plt.show()

The dataset consists of a significant number of positive reviews.

## Count of Reviews by Positive

In [ ]:
df['Positive'].value_counts()

In [ ]:
ax = df['Positive'].value_counts().sort_index().plot(
    kind='bar',
    title='Count of Reviews by Positive',
    figsize=(6, 4)
)
ax.set_xlabel('Positive')
plt.show()

The distribution of data is highly skewed towards positive reviews.

### Word Count by Review Rating

In [ ]:
ax = sns.barplot(df, x='Review Rating', y='Word_count')
ax.set_title('Word count by Review Rating')
plt.show()

Based on the graph, it can be inferred that the number of words decreases as the star rating increases. This is not surprising, as people tend to have more to say when they are dissatisfied.

## Term Frequency

### Corpus
To analyze the most frequent words in the reviews, we will create a function called corpus(). This function will convert the review text into a list of words. However, before doing so, we need to clean the text using the preprocess_text function created earlier using the NLTK library. The cleaning process includes removing punctuations, numbers, single characters, multiple spaces, and stopwords.

In [ ]:
# Empty list where the cleaned text will be stored
clean_text = []

clean_sentences = list(df['Review Text'])
for sen in tqdm(clean_sentences):
    clean_text.append(preprocess_text(sen))

In [ ]:
# A function that takes the review text and splits all the words
def corpus(text):
    test_list = text.split()
    return test_list

# Convert the clean_text list to a pandas Series object
clean_text_series = pd.Series(clean_text)

# Applies the corpus function on the review text and then saves into a new column
df['Corpus_Reviews'] = clean_text_series.apply(corpus)
df.head()

## Frequently occuring N_grams
Most frequently occurring n-grams
What is an n-gram? An n-gram is a sequence of n words in a text. Most words by themselves may not present the entire context. Typically, adverbs such as 'most' or 'very' are used to modify verbs and adjectives. Therefore, n-grams help analyze phrases and not just words, which can lead to better insights.

Some examples of n-grams are:

- bi-gram, which means two words in a sequence, such as 'very good' or 'too great'.
- tri-gram, which means three words in a sequence, such as 'how was your day'. This would be broken down to 'how was your' and 'was your day'.
- quad-gram, which means four words in a sequence.
- quint-gram, which means five words in a sequence.

### Top 15 Words

In [ ]:
# Create an empty list to store all words in the corpus
all_words = []

# Iterate over each review in the corpus and add all words to the all_words list
for review in df['Corpus_Reviews']:
    all_words.extend(review)
    
# Count the frequency of each word and get the 10 most common words
word_freq = Counter(all_words)
top_words = word_freq.most_common(15)

# Print the 10 most common words
print("The 10 most frequent words in the corpus are:")
for word, freq in top_words:
    print(f"{word}: {freq}")

In [ ]:
word_freq = Counter(all_words)
top_words = word_freq.most_common(15)

sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_words], y=[val[0] for val in top_words], color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Word')
plt.show()

### Bi-gram

In [ ]:
# Create an empty list to store all bi-grams in the corpus
all_bigrams = []

# Iterate over each review in the corpus and add all bi-grams to the all_bigrams list
for review in df['Corpus_Reviews']:
    bigrams = ngrams(review, 2)
    all_bigrams.extend(list(bigrams))

# Count the frequency of each bi-gram and get the 10 most common bi-grams
bigram_freq = Counter(all_bigrams)
top_bigrams = bigram_freq.most_common(15)

# Print the 15 most common bi-grams
print("The 15 most frequent bi-grams in the corpus are:")
for bigram, freq in top_bigrams:
    print(f"{bigram}: {freq}")

In [ ]:
# Convert the tuples to strings
top_bigrams_str = [f"{bigram[0]} {bigram[1]}" for bigram, freq in top_bigrams]

sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_bigrams], y=top_bigrams_str, color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Bi-gram')
plt.show()

### Tri-gram

In [ ]:
# Create an empty list to store all tri-grams in the corpus
all_trigrams = []

# Iterate over each review in the corpus and add all tri-grams to the all_trigrams list
for review in df['Corpus_Reviews']:
    trigrams = ngrams(review, 3)
    all_trigrams.extend(list(trigrams))
    
# Count the frequency of each tri-gram and get the 10 most common tri-grams
trigram_freq = Counter(all_trigrams)
top_trigrams = trigram_freq.most_common(15)

# Print the 15 most common tri-grams
print("The 15 most frequent tri-grams in the corpus are:")
for trigram, freq in top_trigrams:
    print(f"{trigram}: {freq}")


In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_trigrams], y=[f"{val[0][0]} {val[0][1]} {val[0][2]}" for val in top_trigrams], color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Tri-gram')
plt.show()

### Top 15 Bad Words

In [ ]:
text = ' '.join(clean_text)  # join the list of strings using a space as separator

# Initialize the sentiment analyzer
analyzer = vader.SentimentIntensityAnalyzer()

# Tokenize the text into individual words
words = word_tokenize(text)

# Get the negative words
negative_words = []
for word in tqdm(words):
    score = analyzer.polarity_scores(word)['neg']
    if score > 0.5:  # Threshold for negative sentiment
        negative_words.append(word)
        
word_freq = Counter(negative_words)
top_words = word_freq.most_common(15)

In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_words], y=[val[0] for val in top_words], color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Bad Word')
plt.show()

### Bi-gram of Bad Words

In [ ]:
# Create a bi-gram of the words
bigrams = ngrams(words, 2)

# Convert the bigrams into strings
bigram_strings = [' '.join(bigram) for bigram in bigrams]

# Get the negative bigrams
negative_bigrams = []
for bigram in tqdm(bigram_strings):
    score = analyzer.polarity_scores(bigram)['neg']
    if score > 0.5:  # Threshold for negative sentiment
        negative_bigrams.append(bigram)
bigram_freq = Counter(negative_bigrams)
top_bigrams = bigram_freq.most_common(15)


In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_bigrams], y=[val[0] for val in top_bigrams], color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Bad Bigram')
plt.show()

### Tri-gram of Bad Words

In [ ]:
# Create tri-grams
trigrams = ngrams(words, n=3)

# Convert the trigrams into strings
trigram_strings = [' '.join(trigram) for trigram in trigram]

# Get the negative words
negative_trigrams = []
for trigram in tqdm(trigrams):
    score = analyzer.polarity_scores(' '.join(trigram))['neg']
    if score > 0.5:  # Threshold for negative sentiment
        negative_trigrams.append(' '.join(trigram))
        
        
# Get the top 15 bad tri-negative_trigrams
trigram_freq = Counter(negative_trigrams)
top_trigram = trigram_freq.most_common(15)


In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=[val[1] for val in top_trigram], y=[val[0] for val in top_trigram], color='r')
ax.set_xlabel('Frequency')
ax.set_ylabel('Bad Tri-gram')
plt.show()

# Prepering for model

It's apparent that the reviews vary in lengths, which is problematic. Neural networks cannot accept data of different lengths. Therefore, it is imperative that we standardize the length of each review. To accomplish this, we need to:

* Split data into train test
* Determine a fxied number of word.
* Tokenize the words.
* Pad the text.
* One-hot encode Y
* Reshape data from 2D to 3D

## Split Data into Train Test

In [ ]:
# Cleaning review text which will then be saved to x

x = []

sentences = list(df['Review Text'])
for sen in sentences:
    x.append(preprocess_text(sen))

In [ ]:
#x = df['Review Text']
x = np.array(x)
y = df['Positive']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

print("shape of x_train:", x_train.shape)
print("shape of x_test:", x_test.shape)

print("shape of y_train:", y_train.shape)
print("shape of y_test:", y_test.shape)


## Determine Number of Words

In [ ]:
# Checking mean and std
df["Word_count"].describe()

### Visual Inspection

In [ ]:
sns.displot(df["Word_count"], height=8, aspect=1.5, bins=200)

In [ ]:
plt.figure(figsize=(6, 10))
sns.boxplot(df["Word_count"])
plt.show()

Upon careful examination of both visuals, it becomes apparent that around 350 words represents the optimal maximum word count.

### IQR Method
To ensure accuracy, I will also use the IQR method to verify the validity or proximity of my observation.

In [ ]:
q1 = df["Word_count"].quantile(0.25)
q3 = df["Word_count"].quantile(0.75)
iqr = q3-q1

In [ ]:
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)

upper_limit, lower_limit

TODO: Check this comment

The upper limit, which is 307.5, closely aligns with my observation. Therefore, I will round it up to 350.

## Tokenizing the text. Creates a word-to-index dictionary

Once a word is tokenized, the assigned number represents its frequency within the text. The number 1 corresponds to the most frequently occurring word, and so forth.

In [ ]:
# This is what the words look like before getting tokenized
print(x_train[4])
print(y_train[4])

In [ ]:
# Let's check what our vocabulary size is
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(x_train.copy())

VOCAB_SIZE = len(tokenizer.word_index)
print("Vocabulary size:", VOCAB_SIZE)

In [ ]:
VOCAB_SIZE = 52234

#, lower=True
tokenizer = Tokenizer(VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)

# Providing each token an integer
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
# This is what the words look like after beeing tokenized
print(x_train[4])
print(y_train[4])

## Padding the text

* If a review exceeds 350 words, we will remove the excess words.
* If a review contains less than 350 words, we will append zeros to it until it reaches a length of 750 words.

In [ ]:
# Word lenghts before padding
len(x_train[0]), len(x_train[1]), len(x_train[2]), len(x_train[3]), len(x_train[4])

In [ ]:
# Max length of words
MAXLEN = 350

In [ ]:
x_train = keras.utils.pad_sequences(x_train, padding='post', maxlen=MAXLEN)
x_test = keras.utils.pad_sequences(x_test, padding='post', maxlen=MAXLEN)

In [ ]:
# Word lenghts after padding
len(x_train[0]), len(x_train[1]), len(x_train[2]), len(x_train[3]), len(x_train[4])

In [ ]:
# And this is what it looks like if we take a closer look
x_train[4]

## One-Hot Encoding on y

In [ ]:
# The assigned number is dependent on the type of rating being used, which is currently set to 0 or 1 in this instance.
# However, it is possible to modify this.
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_test[0])

## Resaping the data from 2D to 3D

Recurrent Neural Networks (RNNs) require 3D input because they are designed to operate on sequences of data. The 3D input tensor consists of a batch size, sequence length, and feature dimensions.

* Batch size: The number of sequences in a batch, typically represented as the first dimension of the 3D tensor.
* Sequence length: The number of time steps in each sequence, represented as the second dimension of the 3D tensor.
* Feature dimensions: The number of features or variables in each time step, represented as the third dimension of the 3D tensor.

The 3D input tensor allows RNNs to process a sequence of data over time, where each time step contains a set of features. By using a 3D input, RNNs can learn and remember patterns and dependencies across multiple time steps, which is useful in a variety of applications such as natural language processing, speech recognition, and time series forecasting.

In [ ]:
x_train = np.array(x_train).reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = np.array(x_test).reshape((x_test.shape[0], x_test.shape[1], 1))

print(x_train.shape)
print("Sentences:", x_train.shape[0], "Words:", x_train.shape[1], "TimeSteps:", x_train.shape[2], "\n")
print(x_test.shape)
print("Sentences:", x_test.shape[0], "Words:", x_test.shape[1], "TimeSteps:", x_test.shape[2])

# Model Building

In [ ]:
# NN Model
model = tf.keras.Sequential([
tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=50, input_length=MAXLEN),
tf.keras.layers.LSTM(32, return_sequences=True),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.LSTM(16, return_sequences=True),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(num_classes , activation="sigmoid"),
])

model.summary()

## Weights

We are dealing with imbalanced data. It's important to adjust the weights of the classes to reflect their distribution. This can be done by calculating the class weights and passing them to the model during training.

The class weights can be calculated using various techniques such as inverse frequency, inverse square root frequency, or user-defined weights. Inverse frequency assigns a weight to each class that is inversely proportional to the number of samples in that class. Inverse square root frequency assigns a weight that is the inverse square root of the frequency of the class. User-defined weights allow the user to manually assign weights to each class based on their domain knowledge.

I opted for inverse frequency as you can seee below.

In [ ]:
# Frequency
freq = pd.value_counts(df['Positive'])
freq

In [ ]:
# Inverse frequency
weights = {0: freq.sum() / freq[0], 1: freq.sum() / freq[1]}
weights

## ModelCheckpoint

In [ ]:
# Creating checkpoint that saves the best model
checkpoint = ModelCheckpoint('model/', save_best_only=True)
#, monitor='val_accuracy', mode='max'

# Stop the training if the val_loss does not improve on next 4 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1)

# Creates a csv file and saves the model metrics in it
log_csv = CSVLogger('my_logs.csv', separator=',', append=False)

# A lsit with the 3 instances that will be put into callbacks in model.fit()
callback_lsit = [checkpoint, early_stop, log_csv]

## Training Model

In [ ]:
# Creating an instance of BinaryCrossentropy to pass into model.compile()
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Configure the model for training.
model.compile(optimizer=Adam(learning_rate=(0.0001)), loss=loss_fn, metrics=['accuracy', AUC(name='AUC')])

# Train a model
history = model.fit(x_train, y_train, batch_size=150, epochs=20, validation_split=0.20, class_weight=weights, callbacks=[callback_lsit])

# print the best epoch
print(f"The best epoch is {early_stop.best_epoch}.")

# Model Evaluation

Let's evaluate the model to see how well it did

In [ ]:
best_model = load_model('model/')

test_predictions = (best_model.predict(x_test) > 0.5).astype(int)
print(classification_report(y_test, test_predictions, zero_division=0))

The precision and recall for both classes are high, indicating that the model is able to correctly identify both positive and negative cases. The f1-score is also high for both classes, which suggests that the model has a good balance between precision and recall.

The macro and weighted average scores are also high, which indicates that the model is performing well overall.

In [ ]:
# Model Performance Chart

# accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# Add a marker on best epoch
plt.axvline(x=early_stop.best_epoch, color='r')

plt.show()

# loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

# Add a marker on best epoch
plt.axvline(x=early_stop.best_epoch, color='r')

plt.show()

# AUC
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])

plt.title('model auc')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# Add a marker on best epoch
plt.axvline(x=early_stop.best_epoch, color='r')

plt.show()


TODO: make this a comment instead with f string that gets the score and round it


Looking at this output, the accuracy score for the training set is around 89-90%, while the accuracy score for the validation set is around 90-91%. This suggests that the model is performing well and not overfitting too much to the training data. The loss scores are also decreasing with each epoch, which is a good sign that the model is learning from the data.


Great! The ROC AUC score of my model is 0.901998439517365 which indicates that my model is performing well. A value of 0.5 indicates random guessing, while a value of 1 indicates perfect performance. A score between 0.7 to 0.9 is considered good, while a score above 0.9 is considered excellent. Therefore, I would say my model is performing well in terms of its ability to distinguish between positive and negative samples.

### Confusion Mtrix

In [ ]:
# TODO Check comment.
# Get the predicted labels for the test set
#y_pred = model.predict(x_test)
#y_pred = (y_pred > 0.5)

# Create the confusion matrix
cm = confusion_matrix(y_test.argmax(axis=1), test_predictions.argmax(axis=1))

print(cm)

In [ ]:
cm[0][0]

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

TODO: make this a comment instead with f string that gets the score and rounds it


Based on the output, the confusion matrix has four values - true negatives (TN), false positives (FP), false negatives (FN), and true positives (TP). The matrix is a 2x2 matrix, with the top row representing the actual negatives and the bottom row representing the actual positives. The left column represents the predicted negatives and the right column represents the predicted positives.

Here, the model predicted 321 true negatives and 4705 true positives. It also predicted 276 false positives and 164 false negatives.

# Making Predictions

Next, we will utilize our neural network to predict the sentiment of our own reviews. However, since the reviews are encoded, we need to convert our new reviews into the same encoding format so that the network can process them. To accomplish this, we will load the encodings from the existing dataset and apply them to our new data.

In [ ]:
data = initial_processing_df
data.shape

In [ ]:
data.head()

In [ ]:
# Empty list where the cleaned text will be stored
data_clean_text = []

data_clean_sentences = list(data['Review'])
for sen in tqdm(data_clean_sentences):
    data_clean_text.append(preprocess_text(sen))

In [ ]:
data_tokenized = tokenizer.texts_to_sequences(data_clean_text)
data_padded = keras.utils.pad_sequences(data_tokenized, padding='post', maxlen=MAXLEN)

In [ ]:
best_model = load_model('model/')

# Use the model to make predictions
data_predictions = best_model.predict(data_padded)

# Get the predicted class labels
data_class_predictions = np.argmax(data_predictions, axis=1)

# Create a DataFrame with the predicted class labels
data_predictions_df = pd.DataFrame(data_class_predictions, columns=['LSTM_Prediction_Positive'])

# Concatenate the original DataFrame with the new predictions DataFrame
data = pd.concat([data, data_predictions_df], axis=1)


In [ ]:
# Let's evaluate the model to see how well it did
print(classification_report(data['Positive'], data['LSTM_Prediction_Positive']))

In [ ]:
data

In [ ]:
# function to compare values in two columns and return boolean
def check_cols(row):
    return row['Positive'] == row['LSTM_Prediction_Positive']

In [ ]:
# apply function to each row using the apply method
data['same_values'] = data.apply(check_cols, axis=1)

data

To calculate the percentage of true and false values, we need to divide the count of each by the total count and multiply by 100.Assuming the count is for a single column, we can calculate the percentages as follows:

In [ ]:
same_values = data['same_values'].value_counts()
same_values

In [ ]:
print(f'Percentage of true values: {same_values[0] / (same_values[0] + same_values[1]) * 100:.2f}%')
print(f'Percentage of false values: {same_values[1] / (same_values[0] + same_values[1]) * 100:.2f}%')